# 처음 인수인계 받은 코드

## keypoints17.sh 그대로 활용
경로만 변경하고 나머지 코드는 아무것도 건드리지 않음.

In [6]:
# 잘 돌아가는지 확인용

In [5]:
import subprocess

# 입력: 프레임 이미지가 들어있는 디렉토리
INPUT = "/workspace/nas203/ds_RehabilitationMedicineData/data/김원 보산진 연구/resize/M03VISIT6하지resized_frames"

# 출력: JSON이 저장될 디렉토리 (스크립트에서 MODEL_NAME 하위폴더 자동 생성됨)
OUTPUT = "../data/Patient_data/sapiens_output/M03VISIT6하지resized_JSON"

# keypoints17.sh 실행
try:
    subprocess.run([
        "bash",
        "../sapiens/pose/scripts/demo/local/keypoints17.sh",  # 수정된 스크립트 경로
        INPUT,
        OUTPUT
    ], check=True)
    print(f"[완료] Pose inference 성공!\n결과 경로: {OUTPUT}")
except subprocess.CalledProcessError as e:
    print("[실패] 스크립트 실행 에러 발생")
    print("에러 코드:", e.returncode)
    print("명령어:", e.cmd)
    if e.output:
        print("출력 로그:", e.output.decode("utf-8", errors="ignore"))


Distributing 9386 image paths into 1 jobs.


/opt/conda/lib/python3.10/site-packages/mmengine/utils/package_utils.py:48: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/opt/conda/lib/python3.10/importlib/__init__.py:169: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  _bootstrap._exec(spec, module)


Loads checkpoint by local backend from path: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Loads checkpoint by local backend from path: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/checkpoints/sapiens_0.3b/sapiens_0.3b_coco_best_coco_AP_796.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: head.deconv_layers.1.weight, head.deconv_layers.1.bias, head.deconv_layers.1.running_mean, head.deconv_layers.1.running_var, head.deconv_layers.4.weight, head.deconv_layers.4.bias, head.deconv_layers.4.running_mean, head.deconv_layers.4.running_var, head.conv_layers.1.weight, head.conv_layers.1.bias, head.conv_layers.1.running_mean, head.conv_layers.1.running_var, head.conv_layers.4.weight, head.conv_layers.4.bias, head.conv_layers.4.running_mean, head.conv_layers.4.running_var



  1%|          | 99/9386 [02:20<4:22:45,  1.70s/it]

KeyboardInterrupt: 

  1%|          | 99/9386 [02:22<3:42:37,  1.44s/it]
Traceback (most recent call last):
  File "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/demo/demo_vis.py", line 246, in <module>
    main()
  File "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/demo/demo_vis.py", line 227, in main
    pred_instances = process_one_image(args, image_path, detector, pose_estimator, visualizer)
  File "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/demo/demo_vis.py", line 55, in process_one_image
    pose_results = inference_topdown(pose_estimator, img, bboxes)
  File "/opt/conda/lib/python3.10/site-packages/mmpose/apis/inference.py", line 196, in inference_topdown
    results = model.test_step(batch)
  File "/opt/conda/lib/python3.10/site-packages/mmengine/model/base_model/base_model.py", line 145, in test_step
    return self._run_forward(data, mode='predict')  # type

## MP4 생성
생성된 output을 가지고 MP4 생성.  
keypoints.sh를 돌리면 json이랑 image 가 같이 출력되기 때문에 해당 파일 경로만 넣어주면 빠르게 mp4 생성 가능.

In [10]:
import os                                                      # 경로 결합과 존재 여부 확인을 위해 os 모듈을 임포트합니다.
import re                                                      # 사람 친화 정렬을 위한 정규식 분해에 re 모듈을 임포트합니다.
import cv2                                                     # 이미지 로드와 그리기 및 비디오 인코딩을 위해 OpenCV를 임포트합니다.
import json                                                    # JSON 파싱을 위해 json 모듈을 임포트합니다.
from glob import glob                                          # 와일드카드 파일 수집을 위해 glob 함수를 임포트합니다.
from pathlib import Path                                       # 경로 객체와 상위 디렉터리 생성에 Path 클래스를 임포트합니다.
from tqdm import tqdm                                          # 진행률 표시를 위해 tqdm을 임포트합니다.

# ===== 입력/출력 경로 =====
OUTPUT = "../../../../data/김원 보산진 연구/resize/M01VISIT2상지resized_frames_out/sapiens_1b"  # 이미지와 JSON이 함께 있는 폴더 경로를 지정합니다.
SAVE_VIDEO = "../data/test_patient_image/first_test_image.mp4"  # 출력 비디오 경로에 .mp4 확장자를 명시합니다.

# ===== 파라미터(그림 옵션) =====
KPT_THR        = 0.01                                          # 키포인트 점수 임계값을 지정합니다.
KEYPOINT_COLOR = (0, 255, 0)                                   # 키포인트 점 색을 BGR 기준으로 지정합니다.
SKELETON_COLOR = (255, 0, 0)                                   # 뼈대 선 색을 BGR 기준으로 지정합니다.
RADIUS         = 4                                             # 키포인트 점의 반지름을 픽셀 단위로 지정합니다.
THICKNESS      = 2                                             # 뼈대 선의 두께를 픽셀 단위로 지정합니다.
LINE_TYPE      = cv2.LINE_AA                                   # 안티앨리어싱 라인 타입을 설정합니다.
FPS_OUT        = 30                                            # 출력 비디오의 FPS를 지정합니다.

# ===== 자연 정렬 유틸 =====
def natural_key(s: str):                                       # 파일명을 숫자 인식 자연 정렬하기 위한 키 함수를 정의합니다.
    base = os.path.basename(s)                                 # 경로에서 파일명만 추출합니다.
    return [int(t) if t.isdigit() else t.lower()               # 숫자는 정수로, 문자는 소문자로 변환해 비교 키를 만듭니다.
            for t in re.split(r'(\d+)', base)]                 # 숫자와 문자의 경계 기준으로 분할합니다.

# ===== skeleton 링크 안전 추출 =====
def get_links(meta: dict):                                     # 메타 정보에서 스켈레톤 링크를 안전하게 꺼내는 함수를 정의합니다.
    raw = meta.get("skeleton_links", meta.get("skeleton", [])) # 새 키(skeleton_links) 또는 구 키(skeleton)에서 원본 리스트를 가져옵니다.
    links = []                                                 # 정규화된 링크를 저장할 리스트를 준비합니다.
    for item in raw:                                           # 원본 리스트의 항목을 순회합니다.
        if isinstance(item, dict) and "link" in item:          # 딕셔너리 형태 { "link": [i, j] }를 처리합니다.
            i, j = item["link"]                                # 링크 쌍을 꺼냅니다.
            links.append((int(i), int(j)))                     # 정수 인덱스 쌍으로 변환해 추가합니다.
        elif isinstance(item, (list, tuple)) and len(item) == 2:  # 리스트/튜플 형태 [i, j]를 처리합니다.
            i, j = item                                        # 두 인덱스를 언팩합니다.
            links.append((int(i), int(j)))                     # 정수 인덱스 쌍으로 변환해 추가합니다.
    return links                                               # 정규화된 링크 리스트를 반환합니다.

# ===== 파일 수집 및 매핑 =====
image_files = sorted(                                          # 이미지 파일 목록을 자연 정렬로 수집합니다.
    glob(os.path.join(OUTPUT, "*.jpg")) +                      # JPG 파일을 포함합니다.
    glob(os.path.join(OUTPUT, "*.jpeg")) +                     # JPEG 파일을 포함합니다.
    glob(os.path.join(OUTPUT, "*.png")),                       # PNG 파일을 포함합니다.
    key=natural_key                                            # 자연 정렬 키를 사용합니다.
)
json_files = sorted(                                           # JSON 파일 목록을 자연 정렬로 수집합니다.
    glob(os.path.join(OUTPUT, "*.json")),                      # 폴더 내 모든 JSON을 찾습니다.
    key=natural_key                                            # 자연 정렬 키를 사용합니다.
)
assert image_files, f"이미지 파일이 없습니다: {OUTPUT}"          # 이미지가 하나도 없으면 중단합니다.
assert json_files, f"JSON 파일이 없습니다: {OUTPUT}"             # JSON이 하나도 없으면 중단합니다.

json_map = {Path(p).stem: p for p in json_files}               # 파일명(확장자 제외)을 키로 JSON 경로를 매핑합니다.
meta_probe = next(iter(json_map.values()))                     # 메타 추출을 위해 임의의 첫 JSON 경로를 선택합니다.
with open(meta_probe, "r", encoding="utf-8") as f:             # 선택한 JSON 파일을 엽니다.
    meta = json.load(f).get("meta_info", {})                   # meta_info 블록을 안전하게 읽어옵니다.
skeleton_links = get_links(meta)                               # 안전 추출 함수로 스켈레톤 링크를 얻습니다.

# ===== 비디오 라이터 초기화 =====
first_frame = cv2.imread(image_files[0])                       # 첫 번째 이미지를 로드해 해상도를 확인합니다.
assert first_frame is not None, f"첫 프레임 로드 실패: {image_files[0]}"  # 첫 프레임을 못 읽으면 중단합니다.
h, w = first_frame.shape[:2]                                   # 프레임의 높이와 너비를 얻습니다.
Path(Path(SAVE_VIDEO).parent).mkdir(parents=True, exist_ok=True)  # 출력 비디오 상위 폴더를 미리 생성합니다.
fourcc = cv2.VideoWriter_fourcc(*"mp4v")                       # mp4v 코덱 식별자를 생성합니다.
out = cv2.VideoWriter(SAVE_VIDEO, fourcc, FPS_OUT, (w, h))     # 지정 FPS와 해상도로 비디오 라이터를 생성합니다.
assert out.isOpened(), f"비디오 라이터 오픈 실패: {SAVE_VIDEO}"   # 라이터가 정상적으로 열렸는지 확인합니다.

# ===== 프레임 단위로 처리 =====
for img_path in tqdm(image_files, desc="비디오 생성 중", unit="frame"):  # 이미지 목록을 순회하며 진행률을 표시합니다.
    img = cv2.imread(img_path)                                   # 현재 이미지를 로드합니다.
    if img is None:                                              # 이미지 로드 실패를 점검합니다.
        print(f"[경고] 이미지 로드 실패: {img_path}")              # 경고 메시지를 출력합니다.
        continue                                                 # 실패한 프레임은 건너뜁니다.
    if img.shape[:2] != (h, w):                                  # 현재 이미지의 해상도가 기준과 다르면
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)  # 비디오 해상도에 맞게 리사이즈합니다.

    stem = Path(img_path).stem                                   # 이미지 파일의 이름(확장자 제외)을 가져옵니다.
    jpath = json_map.get(stem)                                   # 동일 이름의 JSON 경로를 조회합니다.
    if jpath is None:                                            # 매칭 JSON이 없을 때를 처리합니다.
        print(f"[경고] 매칭 JSON 없음: {stem}.json")               # 경고 메시지를 출력합니다.
        out.write(img)                                           # JSON이 없어도 원본 프레임을 비디오에 기록합니다.
        continue                                                 # 다음 프레임으로 넘어갑니다.

    try:                                                         # JSON 파싱 시 예외에 대비합니다.
        with open(jpath, "r", encoding="utf-8") as f:            # 매칭 JSON 파일을 엽니다.
            data = json.load(f)                                  # JSON 내용을 파싱합니다.
    except Exception as e:                                       # 예외를 포착합니다.
        print(f"[경고] JSON 파싱 실패: {jpath} ({e})")             # 경고 메시지를 출력합니다.
        out.write(img)                                           # 파싱 실패 시에도 원본 프레임을 기록합니다.
        continue                                                 # 다음 프레임으로 넘어갑니다.

    for person in data.get("instance_info", []):                 # 각 사람 인스턴스를 순회합니다.
        kpts = person.get("keypoints", [])                       # 키포인트 좌표 목록을 가져옵니다.
        scores = person.get("keypoint_scores", [])               # 키포인트 점수 목록을 가져옵니다.
        for (x, y), score in zip(kpts, scores):                  # 점 좌표와 점수를 함께 순회합니다.
            if score > KPT_THR:                                  # 임계값보다 높은 점수만 시각화합니다.
                cv2.circle(img, (int(x), int(y)), RADIUS, KEYPOINT_COLOR, -1, lineType=LINE_TYPE)  # 키포인트 점을 그립니다.
        for (p1, p2) in skeleton_links:                          # 스켈레톤 링크 쌍을 순회합니다.
            if p1 < len(scores) and p2 < len(scores):            # 인덱스 범위를 안전하게 확인합니다.
                if scores[p1] > KPT_THR and scores[p2] > KPT_THR:  # 두 점 모두 임계값을 넘는지 확인합니다.
                    pt1 = (int(kpts[p1][0]), int(kpts[p1][1]))   # 첫 번째 점의 정수 좌표를 계산합니다.
                    pt2 = (int(kpts[p2][0]), int(kpts[p2][1]))   # 두 번째 점의 정수 좌표를 계산합니다.
                    cv2.line(img, pt1, pt2, SKELETON_COLOR, THICKNESS, lineType=LINE_TYPE)  # 두 점 사이에 선을 그립니다.

    out.write(img)                                               # 오버레이된 프레임을 비디오에 기록합니다.

# ===== 리소스 해제 =====
out.release()                                                    # 비디오 라이터를 닫아 파일을 완성합니다.
print(f"비디오 저장 완료: {SAVE_VIDEO}")                         # 완료 메시지를 출력합니다.


비디오 생성 중: 100% 11370/11370 [03:20<00:00, 56.77frame/s]

비디오 저장 완료: ../data/test_patient_image/first_test_image.mp4


In [8]:
import subprocess
from pathlib import Path

# 입력 디렉토리 (frame 이미지 모음)
INPUT = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/test_patient_image/first_test_image/volleyball14s_frames"

# 출력 디렉토리 (JSON 결과 저장)
OUTPUT = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/test_patient_image/first_test_image/volleyball14s_JSON"

# 출력 폴더 생성
Path(OUTPUT).mkdir(parents=True, exist_ok=True)

# JSON-only 버전 demo 스크립트 실행
try:
    subprocess.run([
        "bash",
        "../sapiens/pose/scripts/demo/local/keypoints17.sh",  # JSON-only 버전 사용
        INPUT,
        OUTPUT
    ], check=True)
    print(f"[완료] Pose inference 성공!\n결과 경로: {OUTPUT}")
except subprocess.CalledProcessError as e:
    print("[실패] 스크립트 실행 에러 발생")
    print("에러 코드:", e.returncode)
    print("명령어:", e.cmd)


Distributing 441 image paths into 1 jobs.


/opt/conda/lib/python3.10/site-packages/mmengine/utils/package_utils.py:48: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/opt/conda/lib/python3.10/importlib/__init__.py:169: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  _bootstrap._exec(spec, module)


Loads checkpoint by local backend from path: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Loads checkpoint by local backend from path: /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/checkpoints/sapiens_0.3b/sapiens_0.3b_coco_best_coco_AP_796.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: head.deconv_layers.1.weight, head.deconv_layers.1.bias, head.deconv_layers.1.running_mean, head.deconv_layers.1.running_var, head.deconv_layers.4.weight, head.deconv_layers.4.bias, head.deconv_layers.4.running_mean, head.deconv_layers.4.running_var, head.conv_layers.1.weight, head.conv_layers.1.bias, head.conv_layers.1.running_mean, head.conv_layers.1.running_var, head.conv_layers.4.weight, head.conv_layers.4.bias, head.conv_layers.4.running_mean, head.conv_layers.4.running_var



Pose inference:   3%|▎         | 12/441 [00:56<33:40,  4.71s/it]

KeyboardInterrupt: 

Pose inference:   3%|▎         | 12/441 [01:00<35:56,  5.03s/it]
Traceback (most recent call last):
  File "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/demo/demo_vis.py", line 123, in <module>
    main()
  File "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/demo/demo_vis.py", line 103, in main
    pred_instances = process_one_image(args, image_path, detector, pose_estimator)
  File "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/sapiens/pose/demo/demo_vis.py", line 46, in process_one_image
    pose_results = inference_topdown(pose_estimator, img, bboxes)
  File "/opt/conda/lib/python3.10/site-packages/mmpose/apis/inference.py", line 196, in inference_topdown
    results = model.test_step(batch)
  File "/opt/conda/lib/python3.10/site-packages/mmengine/model/base_model/base_model.py", line 145, in test_step
    return self._run_forward(data, mode='predict')  # typ